In [1]:
import os
import pyBigWig
import numpy as np
from scipy.stats import ks_2samp
import h5py

# Import variables from config_and_print.py
from config_and_print import methy_directory, output_directory, resolutions, filtered_list

# Read the filtered bam list
with open(filtered_list, 'r') as f:
    filtered_prefixes = set(line.strip() for line in f)

# Define the function to compute the third-order cumulant
def compute_third_order_cumulant(matrix):
    try:
        n, m = matrix.shape
        mean_vec = np.mean(matrix, axis=1, keepdims=True)
        centered_matrix = matrix - mean_vec
        m_ijk = np.zeros((n, n, n))

        for i in range(n):
            for j in range(n):
                for k in range(n):
                    m_ijk[i, j, k] = np.mean(centered_matrix[i] * centered_matrix[j] * centered_matrix[k])
        
        m_ij = np.dot(centered_matrix, centered_matrix.T) / m
        m_i = np.mean(centered_matrix, axis=1)

        kappa_ijk = np.zeros((n, n, n))
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    kappa_ijk[i, j, k] = (
                        m_ijk[i, j, k]
                        - (m_i[i] * m_ij[j, k] + m_i[j] * m_ij[i, k] + m_i[k] * m_ij[i, j])
                        + 2 * m_i[i] * m_i[j] * m_i[k]
                    )

        # Normalize by dividing by the standard deviations
        std_vec = np.std(matrix, axis=1, keepdims=True)
        normalized_kappa_ijk = np.zeros((n, n, n))
        for i in range(n):
            for j in range(n):
                for k in range(n):
                    denominator = std_vec[i] * std_vec[j] * std_vec[k]
                    normalized_kappa_ijk[i, j, k] = kappa_ijk[i, j, k] / denominator if denominator != 0 else np.nan

        min_value = np.min(np.nan_to_num(normalized_kappa_ijk))
        if min_value < 0:
            normalized_kappa_ijk -= min_value

        return normalized_kappa_ijk
    except Exception as e:
        print(f"[ERROR] Failed to compute third-order cumulant: {e}")
        return None

# Function to process each bigWig file
def process_bigwig_file(bigwigfile, large_bin_size, small_bin_size, resolution_label, base_directory, prefix):
    bw = pyBigWig.open(bigwigfile)
    chromosomes = bw.chroms()

    for chrom in chromosomes:
        # Ensure chrom_string is always in the format 'chrN'
        if isinstance(chrom, int) or chrom.isdigit():
            chrom_string = f'chr{chrom}'
        else:
            chrom_string = chrom if chrom.startswith('chr') else f'chr{chrom}'

        if chrom_string in [f'chr{i}' for i in range(1, 23)]:
            # Create chromosome-specific directory if it doesn't exist
            chrom_directory = os.path.join(base_directory, chrom_string)
            os.makedirs(chrom_directory, exist_ok=True)

            # Define the filename for the matrix
            new_filename = os.path.join(chrom_directory, f'{prefix}_{chrom_string}.h5')

            # Check if the file already exists
            if os.path.exists(new_filename):
                print(f"File {new_filename} already exists, skipping computation.")
                continue

            chrom_length = chromosomes[chrom]
            large_bins = range(0, chrom_length, large_bin_size)
            num_large_bins = len(large_bins)

            integrals = np.zeros((large_bin_size // small_bin_size, num_large_bins))

            for i, large_bin_start in enumerate(large_bins):
                for j in range(large_bin_size // small_bin_size):
                    small_bin_start = large_bin_start + j * small_bin_size
                    small_bin_end = small_bin_start + small_bin_size

                    # Ensure that the coordinates are within the chromosome boundaries
                    if small_bin_start < 0:
                        small_bin_start = 0
                    if small_bin_end > chrom_length:
                        small_bin_end = chrom_length

                    # Check if the adjusted small_bin_start is still greater than or equal to small_bin_end
                    if small_bin_start >= small_bin_end:
                        continue

                    try:
                        # Get data within the small bin
                        small_bin_data = bw.values(chrom, int(small_bin_start), int(small_bin_end))

                        # Calculate the integral of data in the small bin
                        integral = np.trapz(small_bin_data)

                        # Store the integral in the corresponding list
                        integrals[j, i] = integral
                    except Exception as e:
                        # Print the exception and relevant information
                        print(f"An error occurred for chromosome {chrom}, large bin {i}, small bin {j}: {e}")

            # create your matrices
            num_rows, num_columns = integrals.shape
            p_value_ks_matrix = np.zeros((num_columns, num_columns))
            epsilon = 1e-10  # Define your epsilon value here

            for i in range(num_columns):
                for j in range(i, num_columns):
                    if i == j:
                        p_value_ks_matrix[i, j] = 1.0
                    else:
                        _, ks_p_value = ks_2samp(integrals[:, i], integrals[:, j])

                        p_value_ks_matrix[i, j] = ks_p_value
                        p_value_ks_matrix[j, i] = ks_p_value
                        
                        #p_value_ks_matrix[i, j] = np.log10(ks_p_value + epsilon)
                        #p_value_ks_matrix[j, i] = np.log10(ks_p_value + epsilon)

            # Compute the third-order cumulant
            cumulant_tensor = compute_third_order_cumulant(p_value_ks_matrix) + 1

            if cumulant_tensor is not None:
                # Save the cumulant matrix as a .h5 file
                with h5py.File(new_filename, 'w') as hf:
                    hf.create_dataset('methy_cumulant', data=cumulant_tensor)

# Ensure resolutions is processed as a list of strings
resolutions_list = [res.strip() for res in resolutions.split(',')]

# Iterate through the files in methy_directory
for root, dirs, files in os.walk(methy_directory):
    for file in files:
        if file.endswith('.methy.b37.bw'):
            # Extract the prefix for filtering
            prefix = '.'.join(file.split('.')[:2])
            if prefix in filtered_prefixes:
                bigwigfile = os.path.join(root, file)
                print(f"Processing {bigwigfile}")
                for resolution in resolutions_list:
                    try:
                        res_value, res_label = resolution.split(":")
                        # Create the base directory for the matrices
                        base_directory = os.path.join(output_directory, f'methy_{res_label}_cumulant_dir')
                        os.makedirs(base_directory, exist_ok=True)
                        large_bin_size = int(res_value)
                        small_bin_size = large_bin_size // 100
                        process_bigwig_file(bigwigfile, large_bin_size, small_bin_size, res_label, base_directory, prefix)
                    except ValueError:
                        print(f"Invalid resolution format: {resolution}")


bam_directory='/home/dwk681/workspace/cluster_cells_from_GSE189158_NOMe_HiC/filesFromCluster/bam'
methy_directory='/home/dwk681/workspace/cluster_cells_from_GSE189158_NOMe_HiC/filesFromCluster/bam/methylation/filter_low_qual'
software_directory='../../bin/softwarefiles'
chrom_file='../../bin/softwarefiles/hg19.autosome.chrom.sizes'
fragments_file='../../bin/softwarefiles/hg19_DpnII.txt'
output_directory='../../projects/single_cell_files'
hg19_fa_url='ftp://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.fa.gz'
filtered_list='../../projects/single_cell_files/filtered_bam_list.txt'
schicluster_env='schicluster2'
bisulfite_env='bisulfitehic27'
min_high_quality_reads='250000'
resolutions=1000000:1Mb
impute=True
cluster_compartments=True
cumulant=True
Processing /home/dwk681/workspace/cluster_cells_from_GSE189158_NOMe_HiC/filesFromCluster/bam/methylation/filter_low_qual/sc24.ACTTGA.methy.b37.bw


NameError: name 'cumulant_matrix' is not defined